In [5]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO
import datetime
import os

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")
names = model.names

# Open the video file
video_path = "video_samples\\3.mkv"
cap = cv2.VideoCapture(video_path)
tracker = sv.ByteTrack()
smoother = sv.DetectionsSmoother()


# List to store points for the polygon
polygon_points = []
# Mouse callback function to get the polygon points
def draw_polygon(event, x, y, flags, param):
    global polygon_points
    if event == cv2.EVENT_LBUTTONDOWN:
        polygon_points.append((x, y))
    elif event == cv2.EVENT_RBUTTONDOWN:
        polygon_points = []


# Variables to store the line start and end points
start_point = None
end_point = None
drawing = False

# Mouse callback function to draw a line
def draw_line(event, x, y, flags, param):
    global start_point, end_point, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        # Start drawing the line
        start_point = (x, y)
        drawing = True
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        # Update the end point as the mouse moves
        end_point = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        # Finish drawing the line
        end_point = (x, y)
        drawing = False


# Step 1: Capture the first frame (snapshot) for polygon drawing
ret, snapshot = cap.read()
if not ret:
    print("Failed to capture video")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Resize the snapshot to 720p
snapshot = cv2.resize(snapshot, (1200, 720))

# Show the snapshot and set the mouse callback
cv2.namedWindow('Draw Polygon on Snapshot')
cv2.setMouseCallback('Draw Polygon on Snapshot', draw_polygon)

while True:
    display_frame = snapshot.copy()

    # Draw the polygon while dragging
    for i in range(len(polygon_points) - 1):
        cv2.line(display_frame, polygon_points[i], polygon_points[i + 1], (0, 255, 0), 2)

    cv2.imshow('Draw Polygon on Snapshot', display_frame)

    # Press 'q' to confirm the polygon and proceed to video processing
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyWindow('Draw Polygon on Snapshot')


# ensure the polygon is closed
if len(polygon_points) > 2:
    polygon_points.append(polygon_points[0])

# convert the polygon points to a numpy array
polygon_points = np.array(polygon_points)

def filter_by_polygon(detections, polygon):
    filtered_detections = sv.Detections()
    for cord, class_id, conf, tracker_id in zip(detections.xyxy, detections.class_id, detections.confidence, detections.tracker_id):
        x1, y1, x2, y2 = cord
        x1 = int(x1)
        y1 = int(y1)
        x2 = int(x2)
        y2 = int(y2)
        center = sv.Point((x1 + x2) / 2, (y1 + y2) / 2)
        if cv2.pointPolygonTest(polygon, center, False) >= 0:
            filtered_detections.add_detection(cord, class_id, conf, tracker_id)
    return filtered_detections
# Step 1: Capture the first frame (snapshot) for line drawing
ret, snapshot = cap.read()
if not ret:
    print("Failed to capture video")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Resize the snapshot to 720p
snapshot = cv2.resize(snapshot, (1200, 720))

# Show the snapshot and set the mouse callback
cv2.namedWindow('Draw Line on Snapshot')
cv2.setMouseCallback('Draw Line on Snapshot', draw_line)

# Create a folder to save the detections, remove all the folders within it if it already exists
if os.path.exists('detections'):
    for folder in os.listdir('detections'):
        folder_path = os.path.join('detections', folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                os.remove(os.path.join(folder_path, file))
            os.rmdir(folder_path)
else:
    os.makedirs('detections')
    

while True:
    display_frame = snapshot.copy()

    # Draw the line while dragging
    if start_point and end_point:
        cv2.line(display_frame, start_point, end_point, (0, 255, 0), 2)

    cv2.imshow('Draw Line on Snapshot', display_frame)

    # Press 'q' to confirm the line and proceed to video processing
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyWindow('Draw Line on Snapshot')

# Define a line zone to count the number of objects crossing it
start = sv.Point(start_point[0], start_point[1])
end = sv.Point(end_point[0], end_point[1])
line_zone = sv.LineZone(start=start, end=end, triggering_anchors=[sv.Position.BOTTOM_CENTER])
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)
polygon_zone = sv.PolygonZone(polygon=polygon_points)
box_annotator = sv.BoxAnnotator()
# Pre-compute resize factors to avoid recomputing in the loop
resize_width = 1200
resize_height = 720
width_factor = resize_width / snapshot.shape[1]
height_factor = resize_height / snapshot.shape[0]
tracker_status = {}
# save as pickle
import pickle
with open("trackers.pkl", 'wb') as f:
    pickle.dump(tracker_status, f)
cap = cv2.VideoCapture(video_path)
# Step 2: Process the actual video using the drawn line
frame_counter = 0
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Skip frames for faster processing
    if frame_counter % 1 != 0:
        frame_counter += 1
        continue

    org_frame = frame.copy()

    # Resize the frame to 720p
    frame = cv2.resize(frame, (resize_width, resize_height))

    # Perform YOLO detection and tracking
    results = model(frame, verbose=False, classes=[3])
    if results[0] is not None:
        detections = sv.Detections.from_ultralytics(results[0])
        mask=polygon_zone.trigger(detections)
        detections=detections[mask]
        detections = detections.with_nms(0.8)
        detections = tracker.update_with_detections(detections)
        detections = smoother.update_with_detections(detections)
        crossed_in, crossed_out = line_zone.trigger(detections)
    
        # Process each detection
        for cord, class_id, conf, tracker_id , in_status, out_status in zip(detections.xyxy, detections.class_id, detections.confidence, detections.tracker_id,crossed_in,crossed_out):
            x1, y1, x2, y2 = cord
            # Rescale the coordinates
            x1=int(x1*org_frame.shape[1]/resize_width)
            y1=int(y1*org_frame.shape[0]/resize_height)
            x2=int(x2*org_frame.shape[1]/resize_width)
            y2=int(y2*org_frame.shape[0]/resize_height)

            # Create folder for each tracker and save detections
            tracker_folder = f'detections/{tracker_id}'
            if not os.path.exists(tracker_folder):
                os.makedirs(tracker_folder)

            img_count = len(os.listdir(tracker_folder)) + 1
            
            if in_status:
                if tracker_id not in tracker_status:
                    tracker_status[tracker_id] = "IN"
            if out_status:
                if tracker_id not in tracker_status:
                    tracker_status[tracker_id] = "OUT"
            if tracker_id not in tracker_status:
                with open('trackers.pkl', 'wb') as f:
                    pickle.dump(tracker_status, f)

            cv2.imwrite(f'{tracker_folder}/{datetime.datetime.now().strftime("%d-%m-%Y_%H-%M_")}{img_count}.jpg', org_frame[y1:y2, x1:x2])

        annotated_frame  = box_annotator.annotate(
        scene=frame, detections=detections)
        line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)
        cv2.imshow("YOLOv8 Inference", annotated_frame)

    frame_counter += 1

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()